In [1]:
%run quant_env.ipynb

from utils.auction_helpers import *
from utils.auctions import *
from utils.upstops import *
from utils.plate import *
from utils.data_source import *

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
auth success 


Loading BokehJS ...

Work on data from 2018-01-01 to 2021-01-29
Constants:
stocks(4161):                name list_date
ts_code                      
688981.SH    中芯国际-U  20200716
689009.SH  九号公司-UWD  20201029
normal_stocks(3552):            name list_date
ts_code                  
605222.SH  起帆电缆  20200731
605288.SH  凯迪股份  20200601


In [2]:
%config Completer.use_jedi = False 
%load_ext Cython

BASIC_COLS = ['name', 'open', 'high', 'low', 'close', 'pct_chg', 'amount']
ORI_COLS = ['amp', 'first_time', 'last_time', 'open_times', 'fc_ratio', 'fl_ratio', 'limit']
ADDED_COLS = ['upstop_num', 'conseq_up_num', 'post_up_num', 'up_type']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [5]:
%%cython

# however this is even slower than plain python
cpdef str cf_set_upstop_types(int upstop_num, float open_, float close, float low, float pre_close):
    cdef str up_type
    if upstop_num == 1:
        if low == close:
            up_type = 'Y' # 一字板
        elif open_ == close and open_ > low:
            up_type = 'T' # T 字板
        elif open_ < pre_close:
            up_type = 'L' # 开盘水下拉涨停
        elif open_/pre_close > 1.05 and open_ < close and open_ == low:
            up_type = 'H' # 高开封板无回调
        else:
            up_type = 'O' # other situation
    else:
        up_type = 'N' # 未涨停
    return up_type

In [6]:
%%prun -l 4

# Cython
df.loc[:, 'up_type'] = df[df.limit=='U'].apply(lambda x: cf_set_upstop_types(x.upstop_num, x.open, x.close, x.low, x.pre_close), axis=1)

         866459 function calls (859742 primitive calls) in 0.822 seconds

   Ordered by: internal time
   List reduced from 697 to 4 due to restriction <4>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    92831    0.270    0.000    0.345    0.000 datetimes.py:603(<lambda>)
    92856    0.041    0.000    0.063    0.000 datetimes.py:627(tz)
        4    0.036    0.009    0.381    0.095 {pandas._libs.lib.map_infer}
      821    0.025    0.000    0.028    0.000 {pandas._libs.lib.infer_dtype}